In [2]:
import pandas as pd
import os

import requests
import re
import time
import numpy as np
import subprocess
import random

from bs4 import BeautifulSoup

### itch.io, only crawling "free browser games"

In [3]:
raw_data = []
page_no = 1
crawled_games = set()

url = "https://itch.io/games/free"
while True:
    params = {'page':page_no}
    new_games = set()
    
    #requext next page
    response = requests.get(url, params=params)
    print(response.url)
    
    #parse results
    soup = BeautifulSoup(response.text, 'html.parser')
    games_boxes = soup.find_all('div', class_='game_cell has_cover lazy_images')
    
    if len(games_boxes) == 0:
        #no results
        break
        
    for x in games_boxes:
        #assume url and title are there by default
        gameurl = x.find('div', class_='game_title').find('a')['href']
        gametitle = x.find('div', class_='game_title').find('a').text
        #extract other metadata
        try:
            gametext = x.find('div', class_='game_text').text
        except:
            gametext = 'NO_TEXT'
        try:
            author = x.find('div', class_='game_author').find('a').text
        except:
            author = 'NO_AUTHOR'
        try:
            author_url = x.find('div', class_='game_author').find('a')['href']
        except:
            author_url = 'NO_AUTHOR_URL'
        try:
            game_genre = x.find('div', class_='game_genre'),text
        except:
            game_genre = 'NO_CAT'
        
        #browser, download for mac, download for win, download for linux 
        platforms = []
        game_platforms = x.find('div', class_='game_platform')
        if game_platforms == None:
            continue 
        for plat in game_platforms.find_all('span'):
            try:
                if plat.text=='Play in browser':
                    platforms.append('web')
            except:
                pass
            
            try:
                if "macOS" in plat['title']:
                    platforms.append('mac')
                if "Windows" in plat['title']:
                    platforms.append('win')
                if "Linux" in plat['title']:
                    platforms.append('linux')
            except:
                pass
        
#         print(gameurl, gametitle, gametext[0:10], author, author_url, platforms, game_genre)
        raw_data.append({'game':gametitle, 
                        'url':gameurl, 
                        'platforms':platforms, 
                        'category':game_genre, 
                        'author':author, 
                        'author_url':author_url, 
                        'desc':gametext})
        new_games.add(gametitle)
        
    if new_games - crawled_games == set(): #new results
        #stop increasing page
        break
    crawled_games = crawled_games | new_games #union
    page_no+=1

        
df_res_cat = pd.DataFrame(raw_data)
df_res_cat.to_csv('itch_game_info.csv')


[<a href="/" id="logo"><img height="40" src="https://static.vseigru.net/templates/vseigru/images/new_logo_small_1.jpg" width="240"/></a>,
 <a class="topm border-7" href="/" style="width:130px;background-color:#A2C90A;box-shadow:0px 4px 0 #88ab00;margin-left:6px;margin-right:1px">Игры</a>,
 <a class="topm border-7" href="/igru-dlya-malchikov.html" style="width:200px;background-color:#3CC0DC;box-shadow:0px 4px 0 #00a1c3;">Игры для мальчиков</a>,
 <a class="topm border-7" href="/igry-novye.html" style="width:125px;background-color:#ff4f5b;box-shadow:0px 4px 0 #f10213;">Новые игры</a>,
 <a class="topm border-7" href="/igry-na-dvoikh.html" style="width:140px;background-color:#ffab00;box-shadow:0px 4px 0 #cc8900;">Игры на двоих</a>,
 <a class="top-link" href="/igru-dlya-malchikov.html">Игры для мальчиков</a>,
 <a href="/igry-parkovka.html" title="Игры Парковка"><img alt="Игры Парковка" class="corners_cat" height="52" src="https://static.vseigru.net/images/new-menu/igry-parkovka.jpg" title="И

In [25]:
pd.read_csv('itch_game_info.csv', index_col=0).head().to_csv("itch_game_info_.csv")

,game,url,platforms,category,author,author_url,desc
0,Another Nightmare,https://codylarkin.itch.io/another-nightmare,"['win', 'linux']",NO_CAT,Cody Larkin,https://codylarkin.itch.io,Experience yet another nightmare
1,The Baby In Yellow,https://teamterrible.itch.io/the-baby-in-yellow,"['win', 'linux']",NO_CAT,Team Terrible,https://teamterrible.itch.io,Who is in control?
2,Saberbeast,https://spalato-bros.itch.io/saberbeast,"['win', 'mac']",NO_CAT,Spalato Bros,https://spalato-bros.itch.io,Use heat and sound to avoid death
3,Stone House Orphanage,https://pixelb.itch.io/stone-house-orphanage,['web'],NO_CAT,pixelb,https://pixelb.itch.io,A lone reporter investigates the disappearance...
4,Cut Short (Full Version),https://hengikken.itch.io/cut-shortgame,['win'],NO_CAT,Hengikken,https://hengikken.itch.io,First person horror game
...,...,...,...,...,...,...,...
222325,Mr Cube,https://thezzk.itch.io/mr-cube,['web'],NO_CAT,thezzk,https://thezzk.itch.io,A simple platformer
222326,The last maze,https://hushh.itch.io/the-last-maze,['win'],NO_CAT,Hushh,https://hushh.itch.io,"Maze, task, reward and you are free"
222327,Blind Spot,https://malloryhillary.itch.io/blind-spot,['web'],NO_CAT,MalloryHillary,https://malloryhillary.itch.io,NO_TEXT
222328,Maze,https://maestro089.itch.io/maze,['win'],NO_CAT,maestro089,https://maestro089.itch.io,Ð½ÐµÐ±Ð¾Ð»ÑÑÐ¾Ð¹ Ð»Ð°Ð±Ð¸ÑÐ¸Ð½Ñ\\\a small ...
